## run this cell before starting spark cluster connection (corrects module error)

In [1]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#import seaborn as sns

pd.set_option('display.max_columns', None)

### select the data

In [4]:
df_ps = spark.read.json("/user/sprenner/training_official_april_hourly_proportion_prefetching.json")

In [5]:
df_ps.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  count  dataset  \
0        0            0            0                0   1624        0   

   datasetHash  datasetScope  datasetScopeHash  dataset_version  \
0            0             0                 0                0   

   dataset_versionHash  datatype  datatypeHash  eventType  eventTypeHash  \
0                    0         0             0          0              0   

   eventVersion  eventVersionHash  fileScope  filename  filenameHash  \
0             0                 0          0         0             0   

   filesize_double  hits  hostname  hostnameHash  hour  hourHash  id  \
0                0     0         0             0     0         0   0   

   id_proportion  int_percentage  join_key_april  join_key_lookup  \
0         197902          197902               0             1624   

   max_percentage  prod_step  prod_stepHash  project  projectHash  protocol  \
0          111420          0              0        0            0         0   

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0             0           0               0           0               0   

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0          0            0                0        0            0        0   

   traceIpHash  traceTimeentryUnix  transferStart  uuid  uuidHash  
0            0                   0              0     0         0

In [6]:
df_ps = df_ps.fillna(0,['count', 'int_percentage','max_percentage'])

In [7]:
df_ps.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  count  dataset  \
0        0            0            0                0      0        0   

   datasetHash  datasetScope  datasetScopeHash  dataset_version  \
0            0             0                 0                0   

   dataset_versionHash  datatype  datatypeHash  eventType  eventTypeHash  \
0                    0         0             0          0              0   

   eventVersion  eventVersionHash  fileScope  filename  filenameHash  \
0             0                 0          0         0             0   

   filesize_double  hits  hostname  hostnameHash  hour  hourHash  id  \
0                0     0         0             0     0         0   0   

   id_proportion  int_percentage  join_key_april  join_key_lookup  \
0         197902               0               0             1624   

   max_percentage  prod_step  prod_stepHash  project  projectHash  protocol  \
0               0          0              0        0            0         0   

   protocolHash  remoteSite  remoteSiteHash  run_number  run_numberHash  \
0             0           0               0           0               0   

   scopeHash  stream_name  stream_nameHash  traceId  traceIdHash  traceIp  \
0          0            0                0        0            0        0   

   traceIpHash  traceTimeentryUnix  transferStart  uuid  uuidHash  
0            0                   0              0     0         0

### make buckets

In [8]:
from pyspark.ml.feature import Bucketizer

splits_percentage = [-float("inf"), 81, float("inf")]
bucketizer = Bucketizer(splits=splits_percentage, inputCol="max_percentage", outputCol="bucketedPercentage")
df_ps = bucketizer.transform(df_ps)

splits_hits = [-float("inf"), 2, float("inf")]
bucketizer = Bucketizer(splits=splits_hits, inputCol="hits", outputCol="bucketedHits")
bucketedData = bucketizer.transform(df_ps)
bucketedData.createOrReplaceTempView("bucket")

In [9]:
spark.sql("SELECT bucketedPercentage, count(*) \
            FROM bucket \
            GROUP BY  bucketedPercentage \
            ORDER BY 1 ASC").toPandas()

bucketedPercentage  count(1)
0                 0.0    527559
1                 1.0    249727

In [10]:
spark.sql("SELECT bucketedHits, count(*) \
            FROM bucket \
            GROUP BY  bucketedHits \
            ORDER BY 1 ASC").toPandas()

bucketedHits  count(1)
0           0.0    541829
1           1.0    235457

In [11]:
final_data = bucketedData.withColumn("prefetch_bucket", when(bucketedData["bucketedPercentage"] == 1.0, "prefetch").otherwise("noprefetch"))

In [12]:
final_data = final_data.withColumn("class_bucket", when(final_data["bucketedHits"] == 0.0, "nocache").otherwise("cache"))

In [13]:
final_data.createOrReplaceTempView("final")
spark.sql("SELECT prefetch_bucket, count(*) \
            FROM final \
            GROUP BY  prefetch_bucket \
            ORDER BY 1 ASC").toPandas()

prefetch_bucket  count(1)
0      noprefetch    527559
1        prefetch    249727

In [14]:
spark.sql("SELECT class_bucket, count(*) \
            FROM final \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache    235457
1      nocache    541829

In [15]:
final_data.describe().toPandas()

summary  account           accountHash         clientState  \
0   count   777286                777286              777286   
1    mean     None   1.935684591732567E8                None   
2  stddev     None  1.1361317241087067E9                None   
3     min  aagaard           -2127257360  CannotAuthenticate   
4     max   zzheng            2143592175      SourceNotFound   

        clientStateHash              count  \
0                777286             777286   
1   -9.74016245191591E8  3673.703860612439   
2  2.6212986830087826E8  7419.355144319622   
3           -1922357674                  0   
4            1269532611             199996   

                                             dataset           datasetHash  \
0                                             777286                777286   
1                                               None  -1.072936835947129E7   
2                                               None   1.258675129155887E9   
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r...           -2146605558   
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_g...            2147407155   

  datasetScope      datasetScopeHash dataset_version   dataset_versionHash  \
0       777286                777286          777286                777286   
1         None  -7.552649492442358E8            None  1.3991617429934412E7   
2         None   8.314865501449597E8            None  1.3045708543551397E9   
3    data10_hi           -2057006107     c1130_m1847           -2146991447   
4   mc16_valid            2109319881      x557_m1947            2146405934   

  datatype          datatypeHash eventType eventTypeHash eventVersion  \
0   777286                777286    777286        777286       777286   
1     None   4.947060556405943E8      None    5.937808E7         None   
2     None  1.1700303292632794E9      None           0.0         None   
3      AOD           -2070622869  download      59378080       1.13.2   
4      log            2124512252  download      59378080       1.15.5   

       eventVersionHash   fileScope                            filename  \
0                777286      777286                              777286   
1  -1.992468944589716E9        None                                None   
2   3.073273964746096E8        None                                None   
3           -2044360925   data10_hi    AOD.05371635._000001.pool.root.1   
4            1261702107  mc16_valid  log.13787275._000072.job.log.tgz.1   

           filenameHash       filesize_double               hits  \
0                777286                777286             777286   
1   -248557.46481861244    6.11402028756827E8  2.010655022732945   
2  1.2402658149710774E9  1.2408771809945793E9   2.33759009262949   
3           -2147478094                 304.0                  1   
4            2147482617       1.1991446515E10                 40   

                   hostname           hostnameHash           hour  \
0                    777286                 777286         777286   
1                      None  -1.1804844919942595E8           None   
2                      None    1.228645222705116E9           None   
3    ReCaS-ui-01.cs.infn.it            -2133418459  2018-04-01 00   
4  xenia.nevis.columbia.edu             2129595856  2018-04-30 23   

                hourHash                 id      id_proportion  \
0                 777286             777286             579384   
1  -2.5176970566176414E7           388643.5  370425.3149948911   
2   1.2215909376898775E9  224383.2850061846  228391.3720806993   
3            -2138243719                  1                  2   
4             2144044799             777286             777286   

       int_percentage                                     join_key_april  \
0              777286                                             777286   
1  23.303134238877323                                               None   
2  28.438942532079476                                          

In [16]:
#final_data = final_data.withColumn("fileScopeHash", hash(final_data['fileScope']))

## Bucketing and Hashing End

In [17]:
final_data = final_data.orderBy("traceTimeentryUnix", ascending=True)

In [18]:
final_data.createOrReplaceTempView('final')
final_data = spark.sql("SELECT ROW_NUMBER() OVER (ORDER BY traceTimeentryUnix) as idx, * FROM final")

In [19]:
final_data.createOrReplaceTempView('final')
spark.sql("SELECT * FROM final WHERE idx <= 621828").count()

621828

In [20]:
spark.sql("SELECT * FROM final WHERE idx > 621828").count()

155458

In [23]:
from pyspark.ml.regression import LinearRegression

from pyspark.ml.evaluation import MulticlassClassificationEvaluator


#variante 1
#assembler_hits = VectorAssembler(inputCols=["accountHash","clientStateHash","count","datasetHash","datasetScopeHash","dataset_versionHash",
#                                       "datatypeHash","eventTypeHash","eventVersionHash","filenameHash", "scopeHash","filesize_double","hostnameHash",
#                                       "hourHash","int_percentage","prod_stepHash","projectHash","protocolHash","remoteSiteHash","run_numberHash",
#                                       "stream_nameHash","traceIdHash","traceIpHash","traceTimeentryUnix","transferStart","uuidHash"], 
#                            outputCol="features_hits")

#assembler_percentage = VectorAssembler(inputCols=["accountHash","clientStateHash","count","datasetHash","datasetScopeHash","dataset_versionHash",
#                                       "datatypeHash","eventTypeHash","eventVersionHash","filenameHash", "scopeHash","filesize_double","hostnameHash",
#                                       "hourHash","int_percentage","prod_stepHash","projectHash","protocolHash","remoteSiteHash","run_numberHash",
#                                       "stream_nameHash","traceIdHash","traceIpHash","traceTimeentryUnix","transferStart","uuidHash"], 
#                            outputCol="features_percentage")

#variante 2 (most important features according to Forward Selection
assembler_hits = VectorAssembler(inputCols=["filesize_double","int_percentage","datatypeHash",
                                       "transferStart","hourHash","eventTypeHash"], 
                            outputCol="features_hits")

assembler_percentage = VectorAssembler(inputCols=["int_percentage","count","filesize_double","datatypeHash",
                                       "traceTimeentryUnix","hourHash","eventTypeHash"], 
                            outputCol="features_percentage")

output = assembler_percentage.transform(final_data)
output = assembler_hits.transform(output)


output.createOrReplaceTempView('out')
train_data = spark.sql("SELECT * FROM out WHERE idx <= 621828")
test_data = spark.sql("SELECT * FROM out WHERE idx > 621828")

#train_data, test_data = output.randomSplit([.8,.2],seed=1234)
lr_percentage = LinearRegression(labelCol='max_percentage', featuresCol='features_percentage', predictionCol="prediction_percentage",maxIter=10, regParam=0.3, elasticNetParam=0.8)

model_percentage = lr_percentage.fit(train_data)
predictions_percentage = model_percentage.transform(test_data)

predictions_percentage = predictions_percentage.withColumn("linear_class_percentage",when(predictions_percentage.prediction_percentage > 80, 1.0).otherwise(0.0))
evaluator = MulticlassClassificationEvaluator(labelCol="max_percentage", predictionCol="linear_class_percentage", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_percentage)
print("Test Error Percentage = %g" % (1.0 - accuracy))
predictions_percentage = predictions_percentage.drop('rawPrediction','probability')

lr_hits = LinearRegression(labelCol='bucketedHits', featuresCol='features_hits', predictionCol="prediction_hits",maxIter=10, regParam=0.3, elasticNetParam=0.8)

model_hits = lr_hits.fit(train_data)
predictions_hits_percentage = model_hits.transform(predictions_percentage)
predictions_hits_percentage = predictions_hits_percentage.withColumn("linear_class_hits",when(predictions_hits_percentage.prediction_hits > 0, 1.0).otherwise(0.0))


evaluator = MulticlassClassificationEvaluator(labelCol="hits", predictionCol="linear_class_hits", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_hits_percentage)
print("Test Error Hits = %g" % (1.0 - accuracy))

#Test Error Percentage no forward no bucket: 0.787666
#Test Error Hits no forward no bucket: 386561

#Test Error Percentage forward regression: 0.786856
#Test Error hits forward regression: 0.410606

#Test Error Percentage forward bucketed: 0.19402
#Test Error hits forward bucketed: 0.386477
#Test Error hits forward bucketed REAL( hits >0 dann 1.0): auch 0.386477

Test Error Percentage = 0.786856
Test Error Hits = 0.386477


In [22]:
predictions_hits_percentage = predictions_hits_percentage.withColumn("filesize", predictions_hits_percentage["filesize_double"].cast(IntegerType()))

In [23]:
final_output = predictions_hits_percentage.select('uuid', 'account', 'datasetScope', 'dataset', 'fileScope','filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions_hits_percentage.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction_hits', 'int_percentage','prediction_percentage').sort(col("day").asc())

In [24]:
final_output.count()

155458

In [25]:
final_output.limit(10).toPandas()

uuid   account  datasetScope  \
0  8d160961f13a4effbc0a381a6518d5ca  tofitsch    mc15_14TeV   
1  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
2  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
3  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
4  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
5  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
6  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
7  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
8  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
9  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   

                                             dataset     fileScope  \
0  mc15_14TeV.167796.Sherpa_CT10_WtaunuMassiveCBP...    mc15_14TeV   
1  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
2  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
3  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
4  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
5  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
6  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
7  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
8  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
9  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   

                                    filename eventType         clientState  \
0   DAOD_TRUTH3.13422301._000001.pool.root.1  download                DONE   
1  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
2  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
3  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
4  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
5  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
6  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
7  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
8  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
9  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   

          day  traceTimeentryUnix    filesize  prediction_hits  \
0  2018-04-22        1.524413e+09   236600961         1.306690   
1  2018-04-22        1.524413e+09  1327098000         3.613257   
2  2018-04-22        1.524413e+09  1379641362         3.633716   
3  2018-04-22        1.524413e+09  1379641362         3.633716   
4  2018-04-22        1.524413e+09  1327098000         3.613257   
5  2018-04-22        1.524413e+09  1327098000         3.613257   
6  2018-04-22        1.524413e+09  1398108348         3.640906   
7  2018-04-22        1.524413e+09  1398108348         3.640906   
8  2018-04-22        1.524413e+09  1398108348         3.640906   
9  2018-04-22        1.524413e+09  1398108348         3.640906   

   int_percentage  prediction_percentage  
0             100             107.462070  
1               0              29.557261  
2               0              28.372443  
3               0              29.723977  
4               0              30.207964  
5               0              30.324354  
6               0              28.239976  
7               0              29.591510  
8               0              28.123548  
9               0              31.166648

In [26]:
final_output = final_output.orderBy("traceTimeentryUnix", ascending=True)
final_output.limit(10).toPandas()

uuid   account  datasetScope  \
0  8d160961f13a4effbc0a381a6518d5ca  tofitsch    mc15_14TeV   
1  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
2  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
3  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
4  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
5  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
6  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
7  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
8  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
9  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   

                                             dataset     fileScope  \
0  mc15_14TeV.167796.Sherpa_CT10_WtaunuMassiveCBP...    mc15_14TeV   
1  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
2  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
3  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
4  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
5  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
6  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
7  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
8  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
9  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   

                                    filename eventType         clientState  \
0   DAOD_TRUTH3.13422301._000001.pool.root.1  download                DONE   
1  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
2  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
3  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
4  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
5  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
6  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
7  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
8  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
9  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   

          day  traceTimeentryUnix    filesize  prediction_hits  \
0  2018-04-22        1.524413e+09   236600961         1.306690   
1  2018-04-22        1.524413e+09  1327098000         3.613257   
2  2018-04-22        1.524413e+09  1379641362         3.633716   
3  2018-04-22        1.524413e+09  1379641362         3.633716   
4  2018-04-22        1.524413e+09  1327098000         3.613257   
5  2018-04-22        1.524413e+09  1327098000         3.613257   
6  2018-04-22        1.524413e+09  1398108348         3.640906   
7  2018-04-22        1.524413e+09  1398108348         3.640906   
8  2018-04-22        1.524413e+09  1398108348         3.640906   
9  2018-04-22        1.524413e+09  1398108348         3.640906   

   int_percentage  prediction_percentage  
0             100             107.462070  
1               0              29.557261  
2               0              28.372443  
3               0              29.723977  
4               0              30.207964  
5               0              30.324354  
6               0              28.239976  
7               0              29.591510  
8               0              28.123548  
9               0              31.166648

## get complete amount of data downloaded in time window to calculate good cache size

In [27]:
final_output.createOrReplaceTempView('output')
spark.sql("SELECT SUM(filesize) FROM (SELECT DISTINCT fileScope, fileName, filesize FROM output)").toPandas()
#36398774222862

sum(filesize)
0  36398774222862

In [28]:
#final_output.write.csv('/user/sprenner/log_april_official_hourly_ordered_prefetching80_maxIter10_regParam03_elasticNetParam08_cache01.csv', sep='\t')

## predict downloads for may with trained model

In [29]:
df_mai = spark.read.json("/user/sprenner/training_official_mai_hourly_proportion_prefetching.json")

In [30]:
df_mai.describe().toPandas()

summary   account           accountHash         clientState  \
0   count    589283                589283              589283   
1    mean      None  -7.873822054121874E7                None   
2  stddev      None   1.130364022555489E9                None   
3     min     abell           -2123889579  CannotAuthenticate   
4     max  zmarshal            2140382084      SourceNotFound   

         clientStateHash               count  \
0                 589283              578132   
1  -1.0635692705599144E9  1573.6514117883114   
2    2.754569203849528E8  10628.459825186388   
3            -1913538301                   1   
4              634051254              195454   

                                             dataset           datasetHash  \
0                                             589283                589283   
1                                               None  -3.916131021861822E7   
2                                               None   1.233601492922985E9   
3  data12_8TeV.00200805.physics_MinBias.recon.NTU...           -2147225360   
4  mc16_valid.361034.Pythia8EvtGen_A2MSTW2008LO_m...            2147407155   

  datasetScope      datasetScopeHash dataset_version   dataset_versionHash  \
0       589283                589283          589283                589283   
1         None  -9.356107326362851E8            None  -7.824071159882942E7   
2         None   6.784957656574688E8            None  1.1845690301436508E9   
3  data12_8TeV           -2057006107     c1128_m1842           -2147305504   
4   mc16_valid            2109319881      x563_m1972            2147255052   

  datatype          datatypeHash eventType          eventTypeHash  \
0   589283                589283    589283                 589283   
1     None  2.1895068833955333E8      None             5.937808E7   
2     None   1.074332683260341E9      None  1.5359058170193747E-9   
3      AOD           -2070622869  download               59378080   
4      log            2124512252  download               59378080   

  eventVersion       eventVersionHash    fileScope  \
0       589283                 589283       589283   
1         None  -2.0115120046498694E9         None   
2         None    3.165646873326391E8         None   
3      1.14.10            -2044360925  data12_8TeV   
4       1.16.1             1261702107   mc16_valid   

                             filename          filenameHash  \
0                              589283                589283   
1                                None     684033.7623824207   
2                                None  1.2390276453463624E9   
3    AOD.05341490._000446.pool.root.1           -2147471303   
4  log.14217468._003414.job.log.tgz.1            2147478625   

        filesize_double                hits                  hostname  \
0                589283              589283                    589283   
1  1.4339739956710258E9  2.9973221694839323                      None   
2   2.193139944581193E9  3.8806741048619684                      None   
3                9576.0                   1    ReCaS-ui-01.cs.infn.it   
4       1.1976018886E10                  74  xenia.nevis.columbia.edu   

           hostnameHash           hour              hourHash  \
0                589283         589283                589283   
1  -4.071422388835245E7           None    -3414275.108615046   
2     9.7563849011858E8           None  1.2200577022275486E9   
3           -2138306752  2018-05-01 00           -2140578122   
4            2129595856  2018-05-31 23            2146843273   

                  id       id_proportion      int_percentage  \
0             589283              357853              357853   
1           294642.0     300680.44414606  31.659882689260673   
2  170111.4936769412  163494.92291812558  31.138705245747943   
3                  1                   2                   1   
4             589283              589283                 100   

                                     join_key_lookup  \


In [31]:
df_mai = df_mai.fillna(0,['count', 'int_percentage','max_percentage'])

In [32]:
splits_percentage_mai = [-float("inf"), 81, float("inf")]
bucketizer_mai = Bucketizer(splits=splits_percentage_mai, inputCol="max_percentage", outputCol="bucketedPercentage")
df_mai = bucketizer_mai.transform(df_mai)

splits_hits_mai = [-float("inf"), 2, float("inf")]
bucketizer_mai = Bucketizer(splits=splits_hits_mai, inputCol="hits", outputCol="bucketedHits")
bucketedData_mai = bucketizer.transform(df_mai)
bucketedData_mai.createOrReplaceTempView("bucket_mai")

#df_mai_final = bucketedData_mai.withColumn("fileScopeHash", hash(bucketedData_mai['fileScope']))
df_mai_final = bucketedData_mai.orderBy("traceTimeentryUnix", ascending=True)

In [33]:
#assembler_mai = VectorAssembler(
#    inputCols=["accountHash","clientStateHash","count","datasetHash","datasetScopeHash","dataset_versionHash",
#                "datatypeHash","eventTypeHash","eventVersionHash","filenameHash", "scopeHash","filesize_double","hostnameHash",
#                "hourHash","int_percentage","prod_stepHash","projectHash","protocolHash","remoteSiteHash","run_numberHash",
#                "stream_nameHash","traceIdHash","traceIpHash","traceTimeentryUnix","transferStart","uuidHash"],
#    outputCol="features")

#variante 2 (most important features according to Forward Selection)

assembler_hits_mai = VectorAssembler(inputCols=["filesize_double","int_percentage","datatypeHash",
                                       "transferStart","hourHash","eventTypeHash"], 
                            outputCol="features_hits")

assembler_percentage_mai = VectorAssembler(inputCols=["int_percentage","count","filesize_double","datatypeHash",
                                       "traceTimeentryUnix","hourHash","eventTypeHash"], 
                            outputCol="features_percentage")

output_mai = assembler_percentage_mai.transform(df_mai_final)
output_mai = assembler_hits_mai.transform(output_mai)


In [34]:
predictions_percentage_mai = model_percentage.transform(output_mai)
predictions_percentage_mai = predictions_percentage_mai.withColumn("linear_class_percentage",when(predictions_percentage_mai.prediction_percentage > 80, 1.0).otherwise(0.0))


evaluator = MulticlassClassificationEvaluator(labelCol="max_percentage", predictionCol="linear_class_percentage", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_percentage_mai)
print("Test Error Percentage Mai = %g" % (1.0 - accuracy))
predictions_percentage_mai = predictions_percentage_mai.drop('rawPrediction','probability')


predictions_hits_percentage_mai = model_hits.transform(predictions_percentage_mai)
predictions_hits_percentage_mai = predictions_hits_percentage_mai.withColumn("linear_class_hits",when(predictions_hits_percentage_mai.prediction_hits > 1, 1.0).otherwise(0.0))


evaluator = MulticlassClassificationEvaluator(labelCol="hits", predictionCol="linear_class_hits", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_hits_percentage_mai)
print("Test Error Hits Mai = %g" % (1.0 - accuracy))
#Test Error Percentage Mai bucketed: 0.304268
#Test error hits Mai bucketed: 0.496663

#Test Error Percentage Mai regression: 0.879859
#Test error hits Mai regression: 0.531726

Py4JJavaError: An error occurred while calling o1225.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 152 in stage 114.0 failed 4 times, most recent failure: Lost task 152.3 in stage 114.0 (TID 5407, ithdp1106.cern.ch, executor 6): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (vector) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.util.collection.Externa

In [ ]:
predictions_hits_percentage_mai=predictions_hits_percentage_mai.withColumn("filesize", predictions_hits_percentage_mai["filesize_double"].cast(IntegerType()))

In [ ]:
predictions_hits_percentage_mai = predictions_hits_percentage_mai.drop('filesize_double')

In [ ]:
final_output_mai = predictions_hits_percentage_mai.select('uuid', 'account', 'datasetScope', 'dataset', 'fileScope','filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions_hits_percentage_mai.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction_hits', 'int_percentage','prediction_percentage').sort(col("day").asc())

In [ ]:
final_output_mai = final_output_mai.orderBy("traceTimeentryUnix", ascending=True)

In [ ]:
final_output_mai.createOrReplaceTempView('output_mai')
spark.sql("SELECT SUM(filesize) FROM (SELECT DISTINCT fileScope, fileName, filesize FROM output_mai)").toPandas()
#209595321347926

In [ ]:
final_output_mai.count()

In [ ]:
final_output_mai = final_output_mai.orderBy("traceTimeentryUnix", ascending=True)

In [ ]:
#final_output_mai.write.csv('/user/sprenner/log_mai_official_hourly_ordered_prefetching80_maxIter10_regParam03_elasticNetParam08_cache01.csv', sep='\t')